In [1]:
import os

os.environ['HF_HOME'] = '/home/husein/ssd3'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from transformers import AutoProcessor, Qwen2AudioForConditionalGeneration, AutoConfig, AutoModelForCausalLM
from transformers import TextStreamer
from datasets import Audio
import torch
import librosa

In [3]:
audio_class = Audio(sampling_rate=16000)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")
tokenizer = processor.tokenizer
audio_token = "<|AUDIO|>"
audio_bos_token = "<|audio_bos|>"
audio_eos_token = "<|audio_eos|>"
audio_token_id = processor.tokenizer._convert_token_to_id_with_added_voc('<|AUDIO|>')
pad_token_id = processor.tokenizer.pad_token_id
torch_dtype = torch.bfloat16
min_dtype = torch.finfo(torch_dtype).min
sequence_length = 4096

In [4]:
streamer = TextStreamer(tokenizer)

In [5]:
# !rm adapter_model.safetensors
# !wget https://huggingface.co/malayloraenjoyer/Malaysian-Qwen2-Audio-7B-Instruct-128/resolve/main/adapter_model.safetensors

In [6]:
ori_model = Qwen2AudioForConditionalGeneration.from_pretrained(
    'Qwen/Qwen2-Audio-7B-Instruct', torch_dtype = torch.bfloat16,
).cuda()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
state_dict = ori_model.state_dict()

In [9]:
from safetensors import safe_open

f = safe_open(f"adapter_model.safetensors", framework="pt", device=0)
keys = f.keys()
keys = sorted(list(set([k.split('.lora')[0] for k in keys if '.lora' in k])))

In [10]:
from tqdm import tqdm

for k in tqdm(keys):
    k_ori = k.replace('base_model.model.', '') + '.weight'
    if 'embed_tokens' in k:
        post_A = '.lora_embedding_A'
        post_B = '.lora_embedding_B'
    else:
        post_A = '.lora_A.weight'
        post_B = '.lora_B.weight'
    A = k + post_A
    B = k + post_B
    
    W = state_dict[k_ori]
    if 'embed_tokens' not in k:
        W = W.t()
        
    A = f.get_tensor(A).type(W.dtype)
    B = f.get_tensor(B).type(W.dtype)
    with torch.no_grad():
        W.addmm_(A.t(), B.t(), alpha = 2)

100%|████████████████████████████████████████████████████████████████████████████████████| 226/226 [00:00<00:00, 228.98it/s]


In [21]:
conversation = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "speech/code-website-react.mp3"},
        {"type": "text", "text": "transcribe it in whisper segment timestamps"},
    ]},
]
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios = []
for message in conversation:
    if isinstance(message["content"], list):
        for ele in message["content"]:
            if ele["type"] == "audio":
                audios.append(librosa.load(ele['audio_url'], 
                    sr=processor.feature_extractor.sampling_rate)[0]
                )

with torch.no_grad():
    inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True).to('cuda')
    generate_kwargs = dict(
        max_new_tokens=128,
        top_p=0.95,
        top_k=50,
        temperature=0.1,
        do_sample=True,
        repetition_penalty=1.05,
        streamer=streamer,
        **inputs,
    )
    generation_output = ori_model.generate(**generate_kwargs)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Audio 1: <|audio_bos|><|AUDIO|><|audio_eos|>
transcribe it in whisper segment timestamps<|im_end|>
<|im_start|>assistant
<|startoftranscript|><|ms|><|transcribe|><|0.82|> Maka, memang mana nak<|1.52|><|2.16|> kod<|2.44|><|2.70|> website pakai<|4.04|><|4.14|> React?<|4.56|><|4.56|> (Makna) Maka, memang mana nak kod website pakai<|4.96|><|5.36|> React?<|7.38|><|7.74|> (Makna) Maka, memang mana nak kod website pakai<|5.64|><|5.98|> React?<|6.64|><|7.22|> (Makna) Maka, memang mana nak kod website pakai<|16.12|><|16.36|> React?<|9.46|><|9.84|> (Makna) Maka, memang mana nak kod website pakai<|16.60|><|16.72|> React?<|17.06|><|17.36|> (


In [22]:
conversation = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "speech/code-website-react.mp3"},
    ]},
]
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios = []
for message in conversation:
    if isinstance(message["content"], list):
        for ele in message["content"]:
            if ele["type"] == "audio":
                audios.append(librosa.load(ele['audio_url'], 
                    sr=processor.feature_extractor.sampling_rate)[0]
                )

with torch.no_grad():
    inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True).to('cuda')
    generate_kwargs = dict(
        max_new_tokens=128,
        top_p=0.95,
        top_k=50,
        temperature=0.9,
        do_sample=True,
        repetition_penalty=1.05,
        streamer=streamer,
        **inputs,
    )
    generation_output = ori_model.generate(**generate_kwargs)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Audio 1: <|audio_bos|><|AUDIO|><|audio_eos|>
<|im_end|>
<|im_start|>assistant
Wix adalah platform web yang popular yang membolehkan anda membuat dan mengurus laman web dengan mudah, tidak seperti React yang merupakan kerangka kerja pembangunan aplikasi web.

Untuk membuat laman web menggunakan Wix, ikuti langkah-langkah berikut:

1. Buka halaman Wix dalam pelayar web anda.
2. Klik "Buat Laman Web" untuk mula projek anda.
3. Pilih template yang sesuai untuk laman web anda. Terdapat pelbagai pilihan template tersedia seperti blog, toko, portfolio, dan banyak lagi


In [20]:
conversation = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "speech/graduan.mp3"},
        {"type": "text", "text": "transcribe it in whisper word timestamps"},
    ]},
]
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios = []
for message in conversation:
    if isinstance(message["content"], list):
        for ele in message["content"]:
            if ele["type"] == "audio":
                audios.append(librosa.load(ele['audio_url'], 
                    sr=processor.feature_extractor.sampling_rate)[0]
                )

with torch.no_grad():
    inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True).to('cuda')
    generate_kwargs = dict(
        max_new_tokens=128,
        top_p=0.95,
        top_k=50,
        temperature=0.01,
        do_sample=True,
        repetition_penalty=1.05,
        streamer=streamer,
        **inputs,
    )
    generation_output = ori_model.generate(**generate_kwargs)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Audio 1: <|audio_bos|><|AUDIO|><|audio_eos|>
transcribe it in whisper word timestamps<|im_end|>
<|im_start|>assistant
<|startoftranscript|><|ms|><|transcribe|><|0.82|> Walaupun mempunyai masyarakat yang pelbagai dan pelbagai bahasa,<|4.10|><|4.50|> mengapa sistem pendidikan Malaysia masih<|6.28|><|6.80|> kurang dalam melahirkan graduan yang berdaya saing<|8.72|><|9.18|> di peringkat global?<|11.58|><|11.98|> Di sini, kita ada 20% orang yang tidak belajar. Ini bermakna kita mempunyai 80% orang yang tidak belajar. Ini bermakna kita mempunyai 80% orang yang tidak belajar


In [24]:
conversation = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "speech/kemiskinan.mp3"},
    ]},
]
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios = []
for message in conversation:
    if isinstance(message["content"], list):
        for ele in message["content"]:
            if ele["type"] == "audio":
                audios.append(librosa.load(ele['audio_url'], 
                    sr=processor.feature_extractor.sampling_rate)[0]
                )

with torch.no_grad():
    inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True).to('cuda')
    generate_kwargs = dict(
        max_new_tokens=128,
        top_p=0.95,
        top_k=50,
        temperature=0.9,
        do_sample=True,
        repetition_penalty=1.05,
        streamer=streamer,
        **inputs,
    )
    generation_output = ori_model.generate(**generate_kwargs)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Audio 1: <|audio_bos|><|AUDIO|><|audio_eos|>
<|im_end|>
<|im_start|>assistant
Malaysia adalah sebuah negara berbilang etnik dan berbilang agama dengan populasi yang pelbagai terdiri daripada kumpulan Melayu, Cina, India, dan lain-lain kumpulan minoriti. Walaupun negara ini telah mencapai kemajuan ekonomi yang ketara dalam beberapa dekad kebelakangan ini, ia masih mempunyai salah satu kadar ketidaksamaan pendapatan tertinggi di Asia Tenggara.

Menurut Bank Dunia, setakat 2019, Malaysia mempuny


In [26]:
conversation = [
    {"role": "user", "content": [
        {"type": "audio", "audio_url": "speech/record1.mp3"},
        {"type": "text", "text": "apa isu"},
    ]},
]
text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios = []
for message in conversation:
    if isinstance(message["content"], list):
        for ele in message["content"]:
            if ele["type"] == "audio":
                audios.append(librosa.load(ele['audio_url'], 
                    sr=processor.feature_extractor.sampling_rate)[0]
                )

with torch.no_grad():
    inputs = processor(text=text, audios=audios, return_tensors="pt", padding=True).to('cuda')
    generate_kwargs = dict(
        max_new_tokens=128,
        top_p=0.95,
        top_k=50,
        temperature=0.9,
        do_sample=True,
        repetition_penalty=1.05,
        streamer=streamer,
        **inputs,
    )
    generation_output = ori_model.generate(**generate_kwargs)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Audio 1: <|audio_bos|><|AUDIO|><|audio_eos|>
apa isu<|im_end|>
<|im_start|>assistant
Isu yang dikemukakan oleh Kerajaan Anwar Ibrahim ialah mengenai isu perniagaan dan ekonomi dalam konteks negara ini.<|im_end|>


In [28]:
ori_model.push_to_hub('malaysia-ai/Malaysian-Qwen2-Audio-7B-Instruct')

[2025-02-26 06:44:01,410] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status


model-00003-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.86G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/malaysia-ai/Malaysian-Qwen2-Audio-7B-Instruct/commit/bcf6f54485f2f53ec5592e4b167024c5421c5b2e', commit_message='Upload Qwen2AudioForConditionalGeneration', commit_description='', oid='bcf6f54485f2f53ec5592e4b167024c5421c5b2e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/malaysia-ai/Malaysian-Qwen2-Audio-7B-Instruct', endpoint='https://huggingface.co', repo_type='model', repo_id='malaysia-ai/Malaysian-Qwen2-Audio-7B-Instruct'), pr_revision=None, pr_num=None)

In [29]:
processor.push_to_hub('malaysia-ai/Malaysian-Qwen2-Audio-7B-Instruct')

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.0M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/malaysia-ai/Malaysian-Qwen2-Audio-7B-Instruct/commit/c7730ac11e06bf3597017b448f9db14e0c9367e0', commit_message='Upload processor', commit_description='', oid='c7730ac11e06bf3597017b448f9db14e0c9367e0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/malaysia-ai/Malaysian-Qwen2-Audio-7B-Instruct', endpoint='https://huggingface.co', repo_type='model', repo_id='malaysia-ai/Malaysian-Qwen2-Audio-7B-Instruct'), pr_revision=None, pr_num=None)

In [31]:
ori_model

Qwen2AudioForConditionalGeneration(
  (audio_tower): Qwen2AudioEncoder(
    (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
    (embed_positions): Embedding(1500, 1280)
    (layers): ModuleList(
      (0-31): 32 x Qwen2AudioEncoderLayer(
        (self_attn): Qwen2AudioSdpaAttention(
          (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
          (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=1280, out_features=5120, bias=True)
        (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        (final_layer_norm): Layer